EXTRACT

In [ ]:
import pandas as pd
df = pd.read_csv('DE_daily_sales.csv')
df.info()

TRANSFORM

In [ ]:
# Convert Transaction Date dtype from object into datetime 
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])
df.info()
display(df)

In [ ]:
# Drop rows with empty Price
df_notnull = df.dropna(subset=['Price'])

# compare before after drop rows 
before = df.shape[0]
after = df_notnull.shape[0]
print('Total row (all) : ', before)
print('Total row (non-empty price) : ',after )

In [ ]:
# Drop duplicate rows
df_distinct = df_notnull.drop_duplicates(subset=['Transaction_ID'])
# compare before after drop rows 
before = df_notnull.shape[0]
after = df_distinct.shape[0]
print('Total row (non-empty price) : ', before)
print('Total row (distinct Transaction ID) : ',after )

In [ ]:
# Add column Total Revenue (Quantity * Price)
df_clean = df_distinct.copy()
df_clean['Total_Revenue'] = df_clean['Quantity'] * df_clean['Price']
display(df_clean)

LOAD

In [ ]:
# Export clean data to csv file
df_clean.to_csv('daily_sales_cleaned.csv', index=False)

Create Table in Postgresql using

-- Membuat table daily_sales_cleaned
CREATE TABLE daily_sales_cleaned (
    transaction_id VARCHAR(50),
    product_id INTEGER,
    quantity INTEGER,
    price NUMERIC(10, 2),
    transaction_date DATE,
    total_revenue NUMERIC(10, 2)
);


In [ ]:
# Load to Postgresql
import psycopg2 
import numpy as np 
import psycopg2.extras as extras 
    
def execute_values(conn, df_clean, table): 
  
    tuples = [tuple(x) for x in df_clean.to_numpy()] 
  
    cols = ','.join(list(df_clean.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    print("the dataframe is inserted") 
    cursor.close() 
  
  
conn = psycopg2.connect( 
    database="postgres", user='postgres', password='password', host='localhost', port='5432'
) 

execute_values(conn, df_clean, 'daily_sales_cleaned') 








Data Analysis

In [ ]:
# Monthly Revenue Trend

import matplotlib.pyplot as plt
df_clean = df_distinct.copy()
df_clean['Total_Revenue'] = df_clean['Quantity'] * df_clean['Price']
df_clean.set_index('Transaction_Date', inplace=True)
# Grouping revenue by month
monthly_revenue = df_clean.resample('M')['Total_Revenue'].sum()
# Plotting
plt.figure(figsize=(10, 6))
monthly_revenue.plot(kind='line', color='blue')
plt.title('Monthly Revenue Trend', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue', fontsize=12)
plt.grid()
plt.show()

In [ ]:
# Sales by Day of the Week

import seaborn as sns
df_clean = df_distinct.copy()
df_clean['Total_Revenue'] = df_clean['Quantity'] * df_clean['Price']
df_clean['day_of_week'] = df_clean['Transaction_Date'].dt.day_name()
# Grouping by day of the week
sales_by_day = df_clean.groupby('day_of_week')['Total_Revenue'].sum()
# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x=sales_by_day.index, y=sales_by_day.values, palette='pastel')
plt.title('Sales by Day of the Week', fontsize=16)
plt.xlabel('Day of the Week', fontsize=12)
plt.ylabel('Total Revenue', fontsize=12)
plt.show()

In [ ]:
#Quantity vs Revenue analysis

df_clean = df_distinct.copy()
df_clean['Total_Revenue'] = df_clean['Quantity'] * df_clean['Price']
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['Quantity'], df_clean['Total_Revenue'], alpha=0.5, c='green')
plt.title('Quantity vs. Revenue', fontsize=16)
plt.xlabel('Quantity', fontsize=12)
plt.ylabel('Revenue', fontsize=12)
plt.grid()
plt.show()